## 載入資料集 (預測與True Label)

In [4]:
import json
from rouge_score import rouge_scorer
from rouge import Rouge

In [5]:
def read_json(path):
    with open(path) as f:
        data = json.load(f)
    return data

In [6]:
test_data = read_json("/user_data/MedQA_DG/model/bart-base/train/test.json")
pred_data = read_json("/user_data/MedQA_DG/model/bart-base/train/pred.json")

In [7]:
len(test_data), len(pred_data)

(1273, 1273)

In [10]:
test_data[0]

'Disclose the error to the patient but leave it out of the operative report[EOD]Disclose the error to the patient and put it in the operative report[EOD]Report the physician to the ethics committee[EOD]Refuse to dictate the operative report'

In [9]:
test_data[0].split('[EOD]')

['Disclose the error to the patient but leave it out of the operative report',
 'Disclose the error to the patient and put it in the operative report',
 'Report the physician to the ethics committee',
 'Refuse to dictate the operative report']

In [11]:
pred_data[0]

'</s><s>Tell the attending that he cannot fail to disclose this complication [EOD] Tell the resident that he</s>'

In [13]:
## Options 不一定會生4個
((pred_data[0].replace('</s>','')).replace('<s>','')).split('[EOD]')

['Tell the attending that he cannot fail to disclose this complication ',
 ' Tell the resident that he']

### 直接將 Prediction 與True Label 算 Rouge Score

True_Label = Option1, Option2, Option3, Option4

Pred_Label = Pred1, Pred2, Pred3, Pred4

Score = Rouge(True_Label, Pred_Label)


In [35]:
precision = 0
recall = 0
f1 = 0
max_score = 0

count1 = 0
count2 = 0

rouge = Rouge()
for i in range(len(test_data)):
    true_label, pred_label = test_data[i],pred_data[i]
    true_options = true_label.replace('[EOD]','')
    pred_options = ((pred_label.replace('</s>','')).replace('<s>','')).replace('[EOD]','')

    prec_1 = 0
    recall_1 = 0
    f1_1 = 0

    
    
    rouge_score = rouge.get_scores(true_options,pred_options)
    r1 = rouge_score[0]["rouge-l"]
    p,r,f = r1['p'],r1['r'],r1['f']
    
    prec_1 = max(prec_1,p)
    recall_1 = max(recall_1,r)
    f1_1 = max(f1_1,f)

    if f1_1 > max_score:
        max_score = f1_1

    precision = prec_1 + precision
    recall = recall_1 + recall
    f1 = f1_1 + f1

precision = precision/(len(test_data))
recall = recall/(len(test_data))
f1 = f1/(len(test_data))

print("precision = ",precision)
print("recall = ",recall)
print("f1 = ",f1)
print("max f1 = ",max_score)


precision =  0.0929095597005422
recall =  0.13790517983369502
f1 =  0.10590361348253792
max f1 =  0.78260869073724


### 將 Prediction 與 True Label 分別計算 Rouge Score 取最高
True_Label = Option1, Option2, Option3, Option4

Pred_Label = Pred1, Pred2, Pred3, Pred4

Score = Max( Rouge(Option1, Pred1), Rouge(Option1, Pred2), …, Rouge(Option4, Pred4))


In [33]:
precision = 0
recall = 0
f1 = 0
max_score = 0

count1 = 0
count2 = 0

rouge = Rouge()
for i in range(len(test_data)):
    true_label, pred_label = test_data[i],pred_data[i]
    true_options = true_label.split('[EOD]')
    pred_options = ((pred_label.replace('</s>','')).replace('<s>','')).split('[EOD]')
    
    prec_1 = 0
    recall_1 = 0
    f1_1 = 0

    if len(pred_options) == 4:
        count1 = count1 + 1
    else:
        count2 = count2 + 1

    
    for option in true_options:
        for pred in pred_options:
            rouge_score = rouge.get_scores(option,pred)
            r1 = rouge_score[0]["rouge-l"]
            p,r,f = r1['p'],r1['r'],r1['f']
            
            prec_1 = max(prec_1,p)
            recall_1 = max(recall_1,r)
            f1_1 = max(f1_1,f)

    

    if f1_1 > max_score:
        max_score = f1_1

    precision = prec_1 + precision
    recall = recall_1 + recall
    f1 = f1_1 + f1

precision = precision/(len(test_data))
recall = recall/(len(test_data))
f1 = f1/(len(test_data))

print("precision = ",precision)
print("recall = ",recall)
print("f1 = ",f1)
print("max f1 = ",max_score)


precision =  0.32659067165325867
recall =  0.34185305180984643
f1 =  0.3205582069985968
max f1 =  0.999999995


#### 有296題有符合生出4個選項 977個不滿4個選項

In [26]:
count1

296

In [27]:
count2

977